In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [3]:
conda install -c anaconda gensim


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\ngohu\anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [4]:
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
print(STOPWORDS)

frozenset({'used', 'too', 'being', 'around', 'third', 'after', 'as', 'mine', 'ltd', 'up', 'describe', 'while', 'least', 'between', 'anyone', 'elsewhere', 'cant', 'here', 'hereafter', 'even', 'itself', 'nobody', 'ourselves', 'should', 'anywhere', 'twenty', 'his', 'we', 'anything', 'found', 'within', 'nowhere', 'six', 'thence', 'very', 'sincere', 'nothing', 'alone', 'go', 'doing', 'sixty', 'herself', 'don', 'side', 'so', 'well', 'nevertheless', 'twelve', 'something', 'be', 'thin', 'using', 'may', 'thereupon', 'they', 'serious', 'are', 'several', 'with', 'interest', 'name', 'since', 'for', 'but', 'been', 'do', 'from', 'four', 'it', 'noone', 'find', 'whereupon', 'those', 'beside', 'across', 'until', 'system', 'hereby', 'nor', 'thereafter', 'towards', 'mostly', 'back', 'whereafter', 'beyond', 'sometime', 'etc', 'by', 'say', 'upon', 'make', 'have', 'him', 'become', 'could', 'or', 'during', 'both', 'under', 'whether', 'at', 'get', 'and', 'cannot', 'yet', 'thru', 'other', 'detail', 'latter', '

# Import data

In [5]:
# import Negative review dataset
negative_rating=pd.read_csv('negative.csv')
negative_rating

,unique_id/0,unique_id/1,asin,product_name,product_type/0,product_type/1,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,0743518144:ok~_but_i_think_the_keirsey_temperm...,389.0,0743518144,"Now, Discover Your Strengths: Books: Marcus Bu...",books,books,7 of 8,2.0,Ok~ but I think the Keirsey Temperment Test is...,"September 10, 2006","Jen ""the truthsayer""","DE, USA","This book has its good points. If anything, it..."
1,0895261715:repellent_sale_of_conservativism:e....,622.0,0895261715,The Conservative Mind: From Burke to Eliot: Bo...,books,books,32 of 95,2.0,Repellent Sale of Conservativism,"August 18, 2005",E. David Swan,"South Euclid, Ohio USA",The fatalistic view of Conservativism is expre...
2,0787978949:i_had_a_bad_feeling_about_this!__an...,1974.0,0787978949,Christian Wisdom of the Jedi Masters: Books: D...,books,books,6 of 12,1.0,I had a bad feeling about this! And I was right!,"December 4, 2005",Z. Fu,"New York, NY","I was intrigued by the title, which supposedly..."
3,"055380121X:lost_credability,_quickly!!:chalupa",4423.0,055380121X,The Wisdom of Menopause: Creating Physical and...,books,books,12 of 15,2.0,"Lost Credability, QUICKLY!!","July 10, 2005",Chalupa,Central California,"I admit, I haven't finished this book. A frie..."
4,"0521662559:poorly_written:""dunner""",5927.0,0521662559,The Creation of America: Through Revolution to...,books,books,6 of 22,2.0,Poorly written,"October 16, 2000","""dunner""","Washington, DC USA",I tried reading this book but found it so turg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0801871476:too_narrow-minded_in_its_approach:a...,NaN,0801871476,Living with Rheumatoid Arthritis (Johns Hopkin...,books,NaN,NaN,2.0,Too narrow-minded in its approach,"November 25, 2006",Aspiring to lifelong growth,South Africa,This book provides excellent information about...
996,0786230800:i_didn't_even_finish_it.:,NaN,0786230800,Sometimes I Dream in Italian: Books: Rita Ciresi,books,NaN,3 of 5,2.0,I didn't even finish it.,"March 20, 2003",NaN,NaN,I really didn't enjoy this book. I am half It...
997,0940793903:adequate_starting_book:m._d._thomas...,NaN,0940793903,A Taste of Lebanon: Cooking Today the Lebanese...,books,NaN,4 of 5,2.0,Adequate Starting Book,"October 9, 2002","M. D. Thomas ""ReadingSailor""","New London, CT",This book is a place to start at best. The ma...
998,"0553587072:implausible_world,_zero_character_d...",NaN,0553587072,Industrial Magic: Women of the Otherworld: Boo...,books,NaN,1 of 2,2.0,"Implausible world, zero character development","August 30, 2006",Pop Culture Dropout,Tokyo,I was barely able to finish this book. Armstro...


In [6]:
#import positive review dataset
positive_rating=pd.read_csv('positive (1).csv')
positive_rating

,unique_id/0,unique_id/1,asin,product_name,product_type/0,product_type/1,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,"0375416811:founder_of_chick_lit:erin_oakes_""av...",1521.0,0375416811,"Bridget Jones Diary: Books: Helen Fielding,Tra...",books,books,NaN,5.0,Founder of chick lit,"August 24, 2006","Erin Oakes ""avid reader""",Central PA,"Bridget Jones, modern day woman, brillant and ..."
1,188105201X:excellent_resource_for_principals!:...,3294.0,188105201X,Leadership and the New Science: Learning About...,books,books,3 of 3,5.0,Excellent resource for principals!,"July 6, 1999",ONICKRE@mail.milwaukee.k12.wi.us,"Milwaukee, Wisconsin",I am ordering copies for all 23 middle school ...
2,0634079190:fun_to_play:michael_p._nolan,3398.0,0634079190,Avenue Q - The Musical (Piano/Vocal arrangemen...,books,books,NaN,5.0,Fun to play,"September 15, 2005",Michael P. Nolan,"Tallahassee, FL United States",As a casual piano player and a Broadway fanati...
3,0786423668:what_a_hottie!:elizabeth_m._hill,4967.0,0786423668,Kay Francis: A Passionate Life and Career: Boo...,books,books,12 of 12,5.0,What a hottie!,"May 16, 2006",Elizabeth M. Hill,"Decatur, GA USA",This is one of the best biographies I have eve...
4,1563523302:great_strategy_+_common_sense:amy-j...,5590.0,1563523302,The Millionaire Next Door: The Surprising Secr...,books,books,5 of 6,5.0,Great Strategy + Common Sense,"September 22, 2006","Amy-Joy Perrone ""AJ""","Schaumburg, IL","I read this book many, many years ago on a ver..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0396083374:boyhood_dreams:robert_l._byers,NaN,0396083374,Diary of an Early American Boy: Noah Blake 180...,books,NaN,1 of 1,4.0,Boyhood Dreams,"December 16, 2005",Robert L. Byers,"Pennsylvania, USA","As a boy, I was fascinated by looking at all o..."
996,0912083786:must_have:a_believer,NaN,0912083786,The Mental Game of Baseball: A Guide to Peak P...,books,NaN,1 of 1,5.0,Must Have,"December 11, 2006",A Believer,"St Paul, Mn",This book is as much about life as it is about...
997,082303867X:awesome:,NaN,082303867X,Painting the Spirit of Nature: Books: Maxine M...,books,NaN,3 of 9,5.0,Awesome,"April 22, 2002",NaN,NaN,If you are looking for ideas and techniques th...
998,0590108840:very_pleased:coolslaw,NaN,0590108840,Giving Thanks: A Native American Good Morning ...,books,NaN,1 of 1,5.0,Very Pleased,"November 11, 2005",coolslaw,Wisconsin,This is a simple book to read through for youn...


In [7]:
negative_chosencolumn=negative_rating[['rating','review_text']]
negative_chosencolumn

,rating,review_text
0,2.0,"This book has its good points. If anything, it..."
1,2.0,The fatalistic view of Conservativism is expre...
2,1.0,"I was intrigued by the title, which supposedly..."
3,2.0,"I admit, I haven't finished this book. A frie..."
4,2.0,I tried reading this book but found it so turg...
...,...,...
995,2.0,This book provides excellent information about...
996,2.0,I really didn't enjoy this book. I am half It...
997,2.0,This book is a place to start at best. The ma...
998,2.0,I was barely able to finish this book. Armstro...


In [8]:
positive_chosencolumn=positive_rating[['rating','review_text']]
positive_chosencolumn

,rating,review_text
0,5.0,"Bridget Jones, modern day woman, brillant and ..."
1,5.0,I am ordering copies for all 23 middle school ...
2,5.0,As a casual piano player and a Broadway fanati...
3,5.0,This is one of the best biographies I have eve...
4,5.0,"I read this book many, many years ago on a ver..."
...,...,...
995,4.0,"As a boy, I was fascinated by looking at all o..."
996,5.0,This book is as much about life as it is about...
997,5.0,If you are looking for ideas and techniques th...
998,5.0,This is a simple book to read through for youn...


In [9]:
#merge together all negative and positive reviews
all_ratings=negative_chosencolumn.append(positive_chosencolumn,ignore_index=True)
all_ratings

,rating,review_text
0,2.0,"This book has its good points. If anything, it..."
1,2.0,The fatalistic view of Conservativism is expre...
2,1.0,"I was intrigued by the title, which supposedly..."
3,2.0,"I admit, I haven't finished this book. A frie..."
4,2.0,I tried reading this book but found it so turg...
...,...,...
1995,4.0,"As a boy, I was fascinated by looking at all o..."
1996,5.0,This book is as much about life as it is about...
1997,5.0,If you are looking for ideas and techniques th...
1998,5.0,This is a simple book to read through for youn...


# Data Pre-processing 

In [11]:
# put another column Negative/Positive. If rating =1,2 then Negative. If rating = 4,5 then Positive.

all_ratings['N_or_P'] = all_ratings['rating'].apply(lambda x: 'Negative' if x <= 2 else 'Postive')
all_ratings

,rating,review_text,N_or_P
0,2.0,"This book has its good points. If anything, it...",Negative
1,2.0,The fatalistic view of Conservativism is expre...,Negative
2,1.0,"I was intrigued by the title, which supposedly...",Negative
3,2.0,"I admit, I haven't finished this book. A frie...",Negative
4,2.0,I tried reading this book but found it so turg...,Negative
...,...,...,...
1995,4.0,"As a boy, I was fascinated by looking at all o...",Postive
1996,5.0,This book is as much about life as it is about...,Postive
1997,5.0,If you are looking for ideas and techniques th...,Postive
1998,5.0,This is a simple book to read through for youn...,Postive


In [12]:
#Add another column, 1 means Negative and ) means Positive
all_ratings['1_or_0'] = all_ratings['rating'].apply(lambda x: 1 if x <= 2 else 0)
all_ratings

,rating,review_text,N_or_P,1_or_0
0,2.0,"This book has its good points. If anything, it...",Negative,1
1,2.0,The fatalistic view of Conservativism is expre...,Negative,1
2,1.0,"I was intrigued by the title, which supposedly...",Negative,1
3,2.0,"I admit, I haven't finished this book. A frie...",Negative,1
4,2.0,I tried reading this book but found it so turg...,Negative,1
...,...,...,...,...
1995,4.0,"As a boy, I was fascinated by looking at all o...",Postive,0
1996,5.0,This book is as much about life as it is about...,Postive,0
1997,5.0,If you are looking for ideas and techniques th...,Postive,0
1998,5.0,This is a simple book to read through for youn...,Postive,0


In [13]:
# put the text in lowercase
all_ratings['review_text']=all_ratings['review_text'].str.lower()
all_ratings

,rating,review_text,N_or_P,1_or_0
0,2.0,"this book has its good points. if anything, it...",Negative,1
1,2.0,the fatalistic view of conservativism is expre...,Negative,1
2,1.0,"i was intrigued by the title, which supposedly...",Negative,1
3,2.0,"i admit, i haven't finished this book. a frie...",Negative,1
4,2.0,i tried reading this book but found it so turg...,Negative,1
...,...,...,...,...
1995,4.0,"as a boy, i was fascinated by looking at all o...",Postive,0
1996,5.0,this book is as much about life as it is about...,Postive,0
1997,5.0,if you are looking for ideas and techniques th...,Postive,0
1998,5.0,this is a simple book to read through for youn...,Postive,0


In [14]:
 #perform the contractions to the review
all_ratings['review_text'] = [re.sub(r"won't", 'will not',str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"would't", ' would not',str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"could't", 'could not',str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'d", 'would',str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"can\'t", 'can not',str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"n\'t", ' not', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text']= [re.sub(r"\'re", ' are', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'s", ' is', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'ll", ' will', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'t", ' not', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'ve", ' have', str(x)) for x in all_ratings['review_text']]
all_ratings['review_text'] = [re.sub(r"\'m", ' am', str(x)) for x in all_ratings['review_text']]
all_ratings

,rating,review_text,N_or_P,1_or_0
0,2.0,"this book has its good points. if anything, it...",Negative,1
1,2.0,the fatalistic view of conservativism is expre...,Negative,1
2,1.0,"i was intrigued by the title, which supposedly...",Negative,1
3,2.0,"i admit, i have not finished this book. a fri...",Negative,1
4,2.0,i tried reading this book but found it so turg...,Negative,1
...,...,...,...,...
1995,4.0,"as a boy, i was fascinated by looking at all o...",Postive,0
1996,5.0,this book is as much about life as it is about...,Postive,0
1997,5.0,if you are looking for ideas and techniques th...,Postive,0
1998,5.0,this is a simple book to read through for youn...,Postive,0


In [15]:
# remove non_alpha characters in the text
all_ratings['review_text']=[re.sub(r'[^A-Za-z]+',' ',str(x)) for x in all_ratings['review_text']]
all_ratings

,rating,review_text,N_or_P,1_or_0
0,2.0,this book has its good points if anything it h...,Negative,1
1,2.0,the fatalistic view of conservativism is expre...,Negative,1
2,1.0,i was intrigued by the title which supposedly ...,Negative,1
3,2.0,i admit i have not finished this book a friend...,Negative,1
4,2.0,i tried reading this book but found it so turg...,Negative,1
...,...,...,...,...
1995,4.0,as a boy i was fascinated by looking at all of...,Postive,0
1996,5.0,this book is as much about life as it is about...,Postive,0
1997,5.0,if you are looking for ideas and techniques th...,Postive,0
1998,5.0,this is a simple book to read through for youn...,Postive,0


In [16]:
#remove space in the text
all_ratings['review_text']=[re.sub(r'\s+',' ',str(x)) for x in all_ratings['review_text']]
all_ratings

,rating,review_text,N_or_P,1_or_0
0,2.0,this book has its good points if anything it h...,Negative,1
1,2.0,the fatalistic view of conservativism is expre...,Negative,1
2,1.0,i was intrigued by the title which supposedly ...,Negative,1
3,2.0,i admit i have not finished this book a friend...,Negative,1
4,2.0,i tried reading this book but found it so turg...,Negative,1
...,...,...,...,...
1995,4.0,as a boy i was fascinated by looking at all of...,Postive,0
1996,5.0,this book is as much about life as it is about...,Postive,0
1997,5.0,if you are looking for ideas and techniques th...,Postive,0
1998,5.0,this is a simple book to read through for youn...,Postive,0


In [17]:
#apply stopwords to the text 
all_ratings['processed_text'] = all_ratings.apply(lambda row: remove_stopwords(row['review_text']), axis=1)
all_ratings

,rating,review_text,N_or_P,1_or_0,processed_text
0,2.0,this book has its good points if anything it h...,Negative,1,book good points helps words want supervisor a...
1,2.0,the fatalistic view of conservativism is expre...,Negative,1,fatalistic view conservativism expressed kirk ...
2,1.0,i was intrigued by the title which supposedly ...,Negative,1,intrigued title supposedly links jedi wisdom c...
3,2.0,i admit i have not finished this book a friend...,Negative,1,admit finished book friend recommended having ...
4,2.0,i tried reading this book but found it so turg...,Negative,1,tried reading book turgid poorly written frust...
...,...,...,...,...,...
1995,4.0,as a boy i was fascinated by looking at all of...,Postive,0,boy fascinated looking drawings descriptions d...
1996,5.0,this book is as much about life as it is about...,Postive,0,book life baseball positive enforcement book g...
1997,5.0,if you are looking for ideas and techniques th...,Postive,0,looking ideas techniques wonderful book
1998,5.0,this is a simple book to read through for youn...,Postive,0,simple book read younger children written nati...


In [18]:
# apply Lemmatization 
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ngohu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
all_ratings['processed_text'] = all_ratings.apply(lambda row: lemmatizer.lemmatize(row['review_text']), axis=1)
all_ratings

,rating,review_text,N_or_P,1_or_0,processed_text
0,2.0,this book has its good points if anything it h...,Negative,1,this book has its good points if anything it h...
1,2.0,the fatalistic view of conservativism is expre...,Negative,1,the fatalistic view of conservativism is expre...
2,1.0,i was intrigued by the title which supposedly ...,Negative,1,i was intrigued by the title which supposedly ...
3,2.0,i admit i have not finished this book a friend...,Negative,1,i admit i have not finished this book a friend...
4,2.0,i tried reading this book but found it so turg...,Negative,1,i tried reading this book but found it so turg...
...,...,...,...,...,...
1995,4.0,as a boy i was fascinated by looking at all of...,Postive,0,as a boy i was fascinated by looking at all of...
1996,5.0,this book is as much about life as it is about...,Postive,0,this book is as much about life as it is about...
1997,5.0,if you are looking for ideas and techniques th...,Postive,0,if you are looking for ideas and techniques th...
1998,5.0,this is a simple book to read through for youn...,Postive,0,this is a simple book to read through for youn...


# Perform Train_test_split and Random Forest Classifier

In [21]:
#using Tf-idf and then Random Forest
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer = TfidfVectorizer()
processed_features = vectorizer.fit_transform(all_ratings['processed_text']).toarray()

In [24]:
processed_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
X=processed_features
y=all_ratings[['1_or_0']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True) 

In [58]:
clf=RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

<ipython-input-58-c8e43291f869>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


In [59]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7775


# Calculation of the Positive Caught Rate and the Negative Caught Rate

In [63]:
rating_test=y_test
rating_test['y_pred']=y_pred
rating_test

<ipython-input-63-b5032c485c2e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_test['y_pred']=y_pred


,1_or_0,y_pred
778,1,1
1713,0,0
1244,0,0
733,1,1
998,1,1
...,...,...
1476,0,0
1159,0,0
1035,0,0
1068,0,1


In [64]:
rating_test_negative=rating_test.loc[(rating_test['1_or_0'] == 1)]
rating_test_negative

,1_or_0,y_pred
778,1,1
733,1,1
998,1,1
432,1,1
152,1,0
...,...,...
77,1,0
123,1,1
70,1,1
484,1,1


In [65]:
print("Negative caught:",metrics.accuracy_score(rating_test_negative['1_or_0'],rating_test_negative['y_pred']))

Negative caught: 0.7475728155339806


In [66]:
rating_test_positive=rating_test.loc[(rating_test['1_or_0'] == 0)]
rating_test_positive

,1_or_0,y_pred
1713,0,0
1244,0,0
1933,0,0
1979,0,0
1820,0,0
...,...,...
1476,0,0
1159,0,0
1035,0,0
1068,0,1


In [67]:
print("Positive caught:",metrics.accuracy_score(rating_test_positive['1_or_0'],rating_test_positive['y_pred']))

Positive caught: 0.8092783505154639
